# North Atlantic O2 gapfill
    - Version 1.3
    - "isopycnal" model, no depth information is entered into the ML model
    - Omits month below the surface mixed layer
    - Absolute magnitude of O2 is estimated as a function of T, S, long, lat, and time (year, month)
    - Reads in the data from WOD profiles
    - Separates the data into training and test data (75% - 25%)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sklearn as skl
import gsw
import cartopy.crs as ccrs
from scipy.interpolate import interp1d
import os
import warnings
warnings.filterwarnings('ignore')

### First prepare O2, T, S, stratification data

In [ ]:
# version information
ver = 1.3

In [ ]:
# observational data 
# -------------------------------------------------------------
# Needs adjustment: this is where you write out the output files
# replace "ito" with your username
diro = '/glade/scratch/ito/WOD18_OSDCTD/'
# -------------------------------------------------------------
dirf = '/glade/campaign/univ/ugit0034/EN4/L09_20x180x360/'
dirin = '/glade/campaign/univ/ugit0034/WOD18_OSDCTD/'
fosd='_1x1bin_osd_'
fctd='_1x1bin_ctd_'
fmer='_1x1bin_merged_'
var=['o2','TSN2']
os.system('mkdir -p '+diro)
os.system('mkdir -p '+diro+'temp')

In [ ]:
dwoa=xr.open_dataset('/glade/campaign/univ/ugit0034/woa18/woa18_all_o00_01.nc',decode_times=False)

In [ ]:
# basin mask
dsm=xr.open_dataset('/glade/campaign/univ/ugit0034/wod18/basin_mask_01.nc')
#! cp /glade/work/ito/basin_mask_01.nc /glade/campaign/univ/ugit0034/wod18/.

In [ ]:
# obtain vertical grid
ds=xr.open_dataset(dirin+var[0]+fmer+str(1965)+'.nc')
Z=ds.depth.to_numpy()
Nz=np.size(Z)
print(Nz)
#
# cutoff depth
zoff = 100
#
# select analysis period
# do not change the start year from 1965 (this is when Carpenter 1965 established modern Winkler method)
yrs=np.arange(1965,2011,1)

In [ ]:
#
# Obtain TS data from observational gridded dataset
#
Nyr=np.size(yrs)
Nt=Nyr*12
mon=["%.2d" % i for i in np.arange(1,13,1)]
#
o2=np.zeros((Nt,Nz,180,360))
t=np.zeros((Nt,Nz,180,360))
s=np.zeros((Nt,Nz,180,360))
mld=np.zeros((Nyr,12,180,360))
#
for year in yrs:
    ytmp=year%10
    if ytmp==0:
        print(str(year))
    ds=xr.open_dataset(dirin+var[0]+fmer+str(year)+'.nc')
    tr=12*(year-1965)
    o2[tr:(tr+12),:,:,:]=ds.o2.to_numpy()
    #
    #option 2. use EN4 for training#
    for mn in range(12):
        ds = xr.open_dataset(dirf+'EN4_TSN2_L09_180x360_'+str(year)+mon[mn]+'.nc')
        s[tr+mn,:,:,:]=ds.SA.interp(depth=Z).to_numpy().squeeze()
        t[tr+mn,:,:,:]=ds.CT.interp(depth=Z).to_numpy().squeeze()
        mld[year-1965,mn,:,:]=ds.MLD_125.to_numpy().squeeze()

In [ ]:
mldc=np.mean(mld,axis=0)

In [ ]:
# get coordinates
x=ds.lon.to_numpy()
y=ds.lat.to_numpy()
xi=np.arange(0,360,1)
yi=np.arange(0,180,1)
xx,yy=np.meshgrid(x,y)
xxi,yyi=np.meshgrid(xi,yi)

In [ ]:
# extract mask in the Atlantic basin
mask= dsm.basin_mask.sel(depth=Z)

In [ ]:
# take a look at the data
fig,ax=plt.subplots(3,2)
# select 500m for plotting
kind=[idx for idx,elem in enumerate(Z) if elem==500]
#
mask.isel(depth=kind).plot(ax=ax[0,0],vmin=0,vmax=5,label='basin_mask')
ax[0,1].pcolormesh(x,y,np.nanmean(o2[:,kind[0],:,:],axis=0))
ax[1,0].pcolormesh(x,y,np.nanmean(s[:,kind[0],:,:],axis=0))
ax[1,1].pcolormesh(x,y,np.nanmean(t[:,kind[0],:,:],axis=0))
ax[2,0].pcolormesh(x,y,np.nanmean(mldc,axis=0))
ax[2,1].remove()

In [ ]:
# get basin mask
ma = dsm.basin_mask.sel(depth=Z).to_numpy()

In [ ]:
# climatological mld
mldc1 = np.tile(mldc,[Nyr,1,1])
mld1 = np.tile(mldc1,[Nz,1,1,1])
mld1  = np.transpose(mld1,(1,0,2,3))+zoff

In [ ]:
# make numpy array
doa = o2
dsa = s
dta = t
#
# prepare depth lat lon time data
xx1 = np.tile(xx,[Nt,Nz,1,1])
yy1 = np.tile(yy,[Nt,Nz,1,1])
ztmp = np.tile(Z,[Nt,180,360,1])
zz1 = np.transpose(ztmp,[0,3,1,2])
t0=np.arange(0,Nt,1)
t1=t0%12
#
tt0 = np.tile(t0,[Nz,180,360,1])
tt1 = np.transpose(tt0,(3,0,1,2))
tc0 = np.tile(t1,[Nz,180,360,1])
tc1 = np.transpose(tc0,(3,0,1,2))

In [ ]:
# ----------------------------------
#  apply mask 
#  basin mask is 1 for Atlantic, 2 for Pacific
# ----------------------------------
doa=np.where((ma==1)&(yy>0),doa,np.nan)
dsa=np.where((ma==1)&(yy>0),dsa,np.nan)
dta=np.where((ma==1)&(yy>0),dta,np.nan)

### Now we have the data for the North Atlantic. Remove NaNs and organize them into 1d array

In [ ]:
# create data vector
doa0 = doa.flatten()
dsa0 = dsa.flatten()
dta0 = dta.flatten()
xx0  = xx1.flatten()
yy0  = yy1.flatten()
tt0  = tt1.flatten()
tc0  = tc1.flatten()
zz0  = zz1.flatten()
ml0  = mld1.flatten()

In [ ]:
# remove nan
dd = doa0 + dsa0 + dta0
doa1=doa0[np.isnan(dd)==False]
dsa1=dsa0[np.isnan(dd)==False]
dta1=dta0[np.isnan(dd)==False]
xx1=xx0[np.isnan(dd)==False]
yy1=yy0[np.isnan(dd)==False]
zz1=zz0[np.isnan(dd)==False]
tt1=tt0[np.isnan(dd)==False]
tc1=tc0[np.isnan(dd)==False]
ml1=ml0[np.isnan(dd)==False]
# clim MLD + cutoff, deeper layer is set to 2 = March
tc1=np.where(zz1<ml1,tc1,2)
#
Nsample = np.size(doa1)
print(Nsample)

### This is where we choose what variables to include

In [ ]:
# generate data matrix and standardize it
X = np.array([dsa1, dta1, xx1, yy1, tt1, tc1])
#X = np.array([dsa1, dta1, xx1, yy1, zz1, tt1, tc1])
y = doa1
#
Xm = np.mean(X,axis=1)
Xstd = np.std(X,axis=1)
#
N=np.size(y)
# normalize x and y
Xa = (X.T - Xm)/Xstd
ym = np.mean(y)
ystd = np.std(y)
ya = (y-ym)/ystd

## ML

### First split the data into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xa, ya)

### Random Forest Regressor (RFregr)

In [ ]:
##
## RF
##
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
#RFregr = RandomForestRegressor()
#RFregr.fit(X_train, y_train)
clf=RandomForestRegressor(n_jobs=10)
parameters = {'n_estimators':[500],'min_samples_leaf':[12],'max_features':[3]}
RFregr = GridSearchCV(clf, parameters)
RFregr.fit(X_train, y_train)

In [ ]:
RFregr.cv_results_

In [ ]:
out = RFregr.predict(X_test)
RFregr.score(X_test, y_test)
#RFregr.get_params()

In [ ]:
o2est = out*ystd + ym
o2test= y_test*ystd + ym
#
plt.plot(o2test,o2est,'k.',markersize=.3)
plt.plot([0,400],[0,400])
plt.ylabel('O2 prediction, micro-mol/kg')
plt.xlabel('O2 test data, micro-mol/kg')
temp=np.corrcoef(o2est,o2test)
Rval_RF=temp[0,1]
print('R2 = ',Rval_RF**2)
RMSE_RF=np.sqrt(np.mean((o2est-o2test)**2))
print('RMSE = ',RMSE_RF)
#
o2test_RF = o2test.copy()
o2est_RF = o2est.copy()

### Neural Network estimator (NNregr)

In [ ]:
## NN
from sklearn.neural_network import MLPRegressor

In [ ]:
#NNregr = MLPRegressor(hidden_layer_sizes=(10,10,10,10),alpha=0.01, 
#                      random_state=1, max_iter=1000).fit(X_train, y_train)
clf=MLPRegressor(max_iter=1000)
parameters = {'hidden_layer_sizes':[[10,10,10,10]],'alpha':[.01]}
NNregr = GridSearchCV(clf, parameters)
NNregr.fit(X_train, y_train)

In [ ]:
NNregr.cv_results_

In [ ]:
out = NNregr.predict(X_test)
NNregr.score(X_test, y_test)

In [ ]:
o2est = out*ystd + ym
o2test= y_test*ystd + ym
#
plt.plot(o2test,o2est,'k.',markersize=.3)
plt.plot([0,400],[0,400])
plt.ylabel('O2 prediction, micro-mol/kg')
plt.xlabel('O2 test data, micro-mol/kg')
temp=np.corrcoef(o2est,o2test)
Rval_NN=temp[0,1]
print('correlation = ',Rval_NN)
RMSE_NN=np.sqrt(np.mean((o2est-o2test)**2))
print('RMSE = ',RMSE_NN)
o2test_NN = o2test.copy()
o2est_NN = o2est.copy()

In [ ]:
# ---------------------------------------------------
# Uncomment the line below to save the final version
# ---------------------------------------------------
import joblib
# comment these out if you don't want to save the results
np.savez(f'ML_params_v{ver}.npz',Xm=Xm,Xstd=Xstd,ym=ym,ystd=ystd)
#
filename = f'NN_model_v{ver}.sav'
joblib.dump(NNregr, filename)
#
filename = f'RF_model_v{ver}.sav'
joblib.dump(RFregr, filename)
#
np.save('mldc.npy',mldc)
np.savez(f'o2test_pred_v{ver}.npz',Xtest=X_test,NNtest=o2test_NN,NNest=o2est_NN,RFtest=o2test_RF,RFest=o2est_RF)

In [ ]:
RFregr.best_estimator_